# Deep dive into the Qumulo API Python Bindings

This notebook is intended to help a user explore the Qumulo API python bindings. First, it will list out all of the supported Qunmulo API python functions. **Don't forget to run <code>pip install qumulo_api</code> first.**

After listing all of the supported python functions, it demonstrates one (ore more) functions inside each of python modules. All demonstration code is read only.

In [1]:
import qumulo
import os
import glob
import re
import time
from datetime import datetime
import dateutil.parser as date_parser
from qumulo.rest_client import RestClient

In [2]:
%%javascript
// this will prevent the large output window below from being boxed in.
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
# set your environment variables or fill in the variables below
API_HOSTNAME = os.environ['API_HOSTNAME'] or '{your-cluster-hostname}'
API_USER = os.environ['API_USER'] or '{api-cluster-user}'
API_PASSWORD = os.environ['API_PASSWORD'] or '{api-cluster-password}'

### Inspect all of the Qumulo python bindings and show methods

In [4]:
qumulo_lib_path = os.path.dirname(qumulo.__file__) + '/rest'

total_matches = 0
for f in glob.glob(qumulo_lib_path + '/*.py'):
    file_name = os.path.basename(f)
    if file_name == '__init__.py':
        continue
    print ""
    print "-"*80
    print "Area: %s" % (file_name, )
    c = open(f, 'r').read()
    rx_str = '@request.request[ \r\n]+def ([^(]+)\([ \r\n]*conninfo,[ \r\n]*credentials([^\)]*)(.*?)(return|yield)'
    ms = re.findall(rx_str, c, re.S|re.M)
    for m in ms:
        total_matches += 1
        func_name = m[0]
        
        # get arguments
        args = []
        arg_ms = m[1].split(',')
        for arg_m in arg_ms:
            if arg_m.strip() != "":
                args.append(re.sub('=.*', '', arg_m.strip()))

        # method
        method = "GET"
        method_m = re.search('method[ ]*=[ ]*"([A-Z]+)', m[2])
        if method_m is not None:
            method = method_m.group(1)

        # uri, currently more work for fs methods
        uri = "/"
        uri_m = re.search('uri[ ]*=.*?"([^"]+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)
        uri_m = re.search('uri[ ]*=.*?\'([^\']+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)

        print "   rc.%s.%s(%s)" % (file_name.replace('.py', ''), 
                                   func_name, 
                                   ', '.join(args[:4]) + (' ...' if len(args)>4 else ''))



--------------------------------------------------------------------------------
Area: ad.py
   rc.ad.list_ad()
   rc.ad.poll_ad()
   rc.ad.join_ad(domain, username, password, ou ...)
   rc.ad.leave_ad(domain, username, password)
   rc.ad.cancel_ad()
   rc.ad.uid_to_sid_get(uid)
   rc.ad.sid_to_uid_get(sid)
   rc.ad.sid_to_gid_get(sid)
   rc.ad.gid_to_sid_get(gid)
   rc.ad.sid_to_expanded_group_sids_get(sid)
   rc.ad.clear_cache_post()

--------------------------------------------------------------------------------
Area: analytics.py
   rc.analytics.time_series_get(begin_time)
   rc.analytics.iops_get(specific_type)
   rc.analytics.current_activity_get(specific_type)
   rc.analytics.capacity_history_get(interval, begin_time, end_time)
   rc.analytics.capacity_history_files_get(timestamp)

--------------------------------------------------------------------------------
Area: auth.py
   rc.auth.login(username, password)
   rc.auth.change_password(old_password, new_password)
   rc.auth.

### Create a new python REST client instance and login.

In [5]:
# Create a new reset client and login.
rc = RestClient(API_HOSTNAME, 8000)
rc.login(API_USER, API_PASSWORD)

### Show Qumulo API python client library version and Qumulo cluster software version

In [6]:
# if qumulo_api is installed via pip, this will return its version.
cmd_output = !pip show qumulo_api

# parse out results if "pip show"
pip_version = '! Unknown !'
for line in cmd_output:
    parts = line.split(':')
    if parts[0] == 'Version':
        pip_version = parts[1]

print("Qumulo API python library version: %s" % (pip_version.strip(),))

# get the Qumulo cluster software version via the API
cluster_version = rc.version.version()
print("Qumulo Cluster software version: %s" % (cluster_version['revision_id'].replace('Qumulo Core ', ''),))

# How old is the current build on the Qumulo cluster?
build_time = int(date_parser.parse(cluster_version['build_date']).strftime('%s'))
cur_time = time.time()
print("Qumulo Cluster software version is: %d days old." % ((cur_time - build_time)/(60*60*24),))

Qumulo API python library version: 2.6.2
Qumulo Cluster software version: 2.6.2
Qumulo Cluster software version is: 12 days old.


## Active Directory (ad)

In [7]:
# Show the current status of the Cluster's AD relationship
rc.ad.list_ad()

{u'base_dn': u'',
 u'domain': u'',
 u'domain_netbios': u'',
 u'ou': u'',
 u'status': u'NOT_IN_DOMAIN',
 u'use_ad_posix_attributes': False}

## Analytics (analytics)

In [8]:
# Get last minute of metrics from the timeseries data. 
# This data is used on the Qumulo applications dashboard home page.
# Show the average value for the last minute for each series.
data = rc.analytics.time_series_get(begin_time=int(time.time() - 60))
for series in data:
    # skip totals since they are duplicated by the other metrics
    if 'total' in series['id']:
        continue
    print "%22s - %11s" % (series['id'],
                       round(sum(series['values']) / len(series['values']), 1))

        iops.read.rate -         7.5
       iops.write.rate -         2.7
 reclaim.deferred.rate -         0.0
 reclaim.snapshot.rate -         0.0
  throughput.read.rate -      7935.5
 throughput.write.rate -    499155.2
